### Train Test Preparation

####  Import Packages

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

#### Load Data

In [2]:
bbc=r"E:\AI\projects\bbc classification\bbc-fulltext\bbc"
categories=os.listdir(bbc)
categories.remove(categories[3])

In [3]:
data=[]
for cat in categories:
    for file in glob.glob(r"E:\AI\projects\bbc classification\bbc-fulltext\bbc\\"+cat+"\\*.TXT"):
        f=open(file)
        content=f.read()
        filename=file.split("\\")[-1]
        id=cat+filename.split('.')[0]
        data.append([cat,id,content])        

##### Base Data Creation

In [4]:
data_array=np.array(data)
bbc_data=pd.DataFrame(data_array)

#### Renaming Features and Traget

In [5]:
bbc_data.rename({0:'category',1:'id',2:'content'},axis=1,inplace=True)

##### Labelling categorical values

In [6]:
bbc_data['category_id']=bbc_data['category'].factorize()[0]

#### Basic Feature Extraction

##### Extracting title

In [7]:
text = bbc_data["content"].str.split("\n", n = 1, expand = True) 

bbc_data["title"] =  text[0]
bbc_data['content'] = text[1]

#### Text Preprocessing

In [8]:
#Creating a stopword list
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","january","february","march","april","may","june","july","august","septemer",\
             "october","november","december","today","tomorrow","yesterday"]

In [9]:
import re
import unicodedata

def clean_text(text):
    #Treat accented characters
    text=unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8','ignore')    
    
    # Decontracting Words
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text=re.sub(r"They'd", "they would", text)
    text=re.sub(r"don't", "do not", text)
    
    #remove html tags
    pattern=re.compile(r'<.*?>')
    text=re.sub(pattern,' ',text)
    pattern=re.compile(r'http\?s\S+')
    text=re.sub(pattern,' ',text)
    
    # remove Special characters
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = text.lower()

    # remove line breaks \r \n \t remove from string 
    text = text.replace('\\r', ' ')
    text = text.replace('\\"', ' ')
    text = text.replace('\\t', ' ')
    text = text.replace('\\n', ' ')

    # remove stopwords
    text = ' '.join(word.lower() for word in text.split() if word not in stopwords)

    return text

##### title

In [10]:
from tqdm import tqdm
clean_titles = []
for title in tqdm(bbc_data['title'].values):
    clean_title = clean_text(title)
    clean_titles.append(clean_title)
bbc_data['title']=clean_titles

100%|███████████████████████████████████| 2225/2225 [00:00<00:00, 10846.02it/s]


##### content

In [11]:
from tqdm import tqdm
clean_contents = []
for content in tqdm(bbc_data['content'].values):
    clean_content = clean_text(content)
    clean_contents.append(clean_content)
bbc_data['content']=clean_contents

100%|█████████████████████████████████████| 2225/2225 [00:04<00:00, 471.86it/s]


#### Traget-Features Split

In [12]:
X=bbc_data.iloc[:,bbc_data.columns!='category_id']
X.drop(['id','category'],axis=1,inplace=True)
Y=bbc_data['category_id']

#### Train-Test Split

In [13]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.30,stratify=Y)

### Vectorization

#### title

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(ngram_range=(1,2),min_df=3)
tf.fit(xtrain['title'].values)
title_trtf=tf.transform(xtrain['title'].values)
title_tetf=tf.transform(xtest['title'].values)

#### content

In [15]:
tf=TfidfVectorizer(ngram_range=(1,2),min_df=5)
tf.fit(xtrain['content'].values)
content_trtf=tf.transform(xtrain['content'].values)
content_tetf=tf.transform(xtest['content'].values)

### Preparing Data Matrix

In [16]:
from scipy.sparse import  hstack
train=hstack((title_trtf,content_trtf)).tocsr()
test=hstack((title_tetf,content_tetf)).tocsr()

### Base Model

In [17]:
from sklearn.feature_selection import SelectKBest, chi2
kbest=SelectKBest(chi2, k=6000)
kbest.fit(train, ytrain)
train=kbest.transform(train)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
lgr=LogisticRegression()
ovr=OneVsRestClassifier(lgr)
ovr.fit(train,ytrain)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [19]:
import pickle
with open('pickle_train','wb') as f:
    pickle.dump([ovr,xtrain,train,ytrain,kbest],f)